In [1]:
import pandas as pd
import numpy as np

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# https://baeseongsu.github.io/posts/mimiciii/
# 기초 테이블 확인 : https://mimic.mit.edu/docs/iii/tables/
# 스키마 확인 : https://mit-lcp.github.io/mimic-schema-spy/

# 01 폐렴환자 추출

In [41]:
lab.columns
lab.shape

Index(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUE',
       'VALUENUM', 'VALUEUOM', 'FLAG'],
      dtype='object')

(9356349, 9)

In [2]:
patient = pd.read_csv('/project/data/PATIENTS.csv')
lab = pd.read_csv('/project/data/LABEVENTS.csv')
diagnoses = pd.read_csv('/project/data/D_ICD_DIAGNOSES.csv') # 질병 정보만 있음 
diagnoses_icd = pd.read_csv('/project/data/DIAGNOSES_ICD.csv') # 환자별 질병 정보 

In [3]:
# 1. 폐와 관련된 질병 추출 (95가지)
diag_pneu = diagnoses[(diagnoses['SHORT_TITLE'].str.contains('pneum')|(diagnoses['SHORT_TITLE'].str.contains('Pneum')))]
diag_pneu.shape

# 2. 폐와 관련된 질병을 가진 환자 추출 -> 14159명
# 가장 많은 폐렴 종류 상위 3개 -> 486, 5070, 48241 (top3_pneu)
pneu_id = diagnoses_icd[diagnoses_icd['ICD9_CODE'].isin(diag_pneu['ICD9_CODE'])].reset_index()
top3_pneu = pneu_id['ICD9_CODE'].value_counts()[:3].index

# 3. ICD9_CODE : 486, 5070, 48241의 정확한 병명 확인
# -> 486 : 
# -> 5070 : 음식 또는 구토물에 의한 폐렴
# -> 48241 : 
diagnoses[diagnoses['ICD9_CODE'].isin(top3_pneu)]

# 4. top3_pneu에 해당하는 환자 추출 
pneu_sub = pneu_id[(pneu_id['ICD9_CODE'].isin(top3_pneu))].drop(['index','ROW_ID','SEQ_NUM'],axis=1).reset_index(drop=True)

# 5. pneu_sub에 한 환자 당 여러 병원기록을 가지고 있을 수 있음
# 예측에 활용할 unique한 환자 수 : 7807
환자id = pneu_sub['SUBJECT_ID'].unique()
len(환자id)

# 6. 추출한 환자id로 환자의 사망, 생존 분포 확인
# -> 사망 : 4777, 생존 : 3030
patient[patient['SUBJECT_ID'].isin(환자id)]['EXPIRE_FLAG'].value_counts()

# 7. 추출한 환자id로 patient에서 환자 정보 추출
환자 = patient[patient['SUBJECT_ID'].isin(환자id)]
# 환자

# 환자.to_csv('폐렴환자.csv', index=False)

(95, 4)

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
5129,5509,48241,Meth sus pneum d/t Staph,Methicillin susceptible pneumonia due to Staph...
5147,5528,486,"Pneumonia, organism NOS","Pneumonia, organism unspecified"
5407,5136,5070,Food/vomit pneumonitis,Pneumonitis due to inhalation of food or vomitus


7807

1    4777
0    3030
Name: EXPIRE_FLAG, dtype: int64

In [4]:
# 8. labevents에서 폐렴환자 추출 : '환자lab'
환자lab = lab[lab['SUBJECT_ID'].isin(환자id)].reset_index(drop=True)

# 9. 'FLAG'열의 결측치를 'nan'으로 설정
환자lab['FLAG'] = 환자lab['FLAG'].fillna('nan')

# 10. 환자의 labevents에서 item별 flag값 확인 
len(환자lab)
환자lab['FLAG'].value_counts()

# 11. 환자lab의 전체 ITEMID 개수 확인
환자lab['ITEMID'].nunique()

# 12. 환자lab -> csv로 저장 
# 환자lab.to_csv('폐렴환자lab.csv', index=False)

# 13. patient의 폐렴 환자가 labevent의 폐렴 환자수보다 8명 더 많음
# 외래환자는 labevent의 'HADM_ID'를 가지지 않음 
외래환자idx = list(set(환자id) - set(환자lab['SUBJECT_ID'].unique()))
외래환자idx

9356349

nan         5931403
abnormal    3406609
delta         18337
Name: FLAG, dtype: int64

690

[60961, 48968, 17674, 9388, 19097, 93114, 58012, 95230]